### Predicting

In [7]:
train = pd.read_csv('../Data/train.csv')
test = pd.read_csv('../Data/test.csv')

In [277]:
def toKaggle(test,pred):
    

    resp = test.copy()
    resp['SalePrice'] = pred
    resp_f = resp.loc[:,['Id','SalePrice']]
    
    return resp_f

In [8]:
def pre_process(train,test):
    # Mean imputation
    train = train.fillna(train.mean())
    # Other label for categorical
    train = train.fillna('Other')

    # Lets get X and Y
    X_train = train.drop('SalePrice', axis =1)
    y_train = train.SalePrice

    test_X = test.drop('Id',axis = 1)
    # Mean inputation for continuos.
    test_X = test_X.fillna(test_X.mean())
    # Other label for categorical
    test_X = test_X.fillna('Other')
    
    return X_train,y_train,test_X

In [9]:
X_train,y_train,test_X = pre_process(train,test)

### Gradient boosting Gridsearch

In [10]:
GradientBoostingRegressor().get_params().keys()

dict_keys(['alpha', 'criterion', 'init', 'learning_rate', 'loss', 'max_depth', 'max_features', 'max_leaf_nodes', 'min_impurity_decrease', 'min_impurity_split', 'min_samples_leaf', 'min_samples_split', 'min_weight_fraction_leaf', 'n_estimators', 'n_iter_no_change', 'presort', 'random_state', 'subsample', 'tol', 'validation_fraction', 'verbose', 'warm_start'])

In [14]:
def GBoost_w_GS(X_train,y_train):
    
    categorical = X_train.dtypes == object

    # Lets do the column transformer with Standar Scaler and One-Hot Encoder.
    preprocess = make_column_transformer(
        (RobustScaler(), ~categorical),
        (OneHotEncoder(handle_unknown = 'ignore'), categorical))

    model_XGB = make_pipeline(preprocess, GradientBoostingRegressor(alpha = 0.85,n_estimators = 1000,max_depth = 3, max_features = 70))


    # Linear Regression.

    param_grid_XGB = {#'gradientboostingregressor__max_depth': range(1,10,1), #  3.
                      #'gradientboostingregressor__max_features':range(1,80,4)#, # 70
                      #'gradientboostingregressor__alpha':np.logspace(-2, -0.001, num=30) # 0.85
                      }

    grid_XGB = GridSearchCV(model_XGB,param_grid_XGB, cv=20)
    grid_XGB.fit(X_train, y_train)
    
    print('\t\t\t XGBoost: \n')
    print(grid_XGB.best_params_, '\t score: ',grid_XGB.score(X_train, y_train),'\n')
    
    return grid_XGB

In [15]:
grid_GB = GBoost_w_GS(X_train,y_train)

			 XGBoost: 

{} 	 score:  0.9970868178237801 



In [ ]:
y_pred_gb = grid_GB.predict(X_test)

* Tune max_features
* Tune column subsampling, row subsampling
* Typically strong pruning via max_depth
* Regularization
* Tune learning rate and do early stopping?

#### Strategies for tuning: Ranges
* Max depth: He goes 1,2,3. People sometimes go to 20.
* Learning rate: 0.1,0.01,0.001,etc
* Regularization: Logarithmic scale

### Light GBM

In [2]:
def LGBM(X_train,y_train):

    categorical = X_train.dtypes == object

    # Lets do the column transformer with Standar Scaler and One-Hot Encoder.

    preprocess = make_column_transformer(
        (RobustScaler(), ~categorical),
        (OneHotEncoder(handle_unknown = 'ignore'), categorical))

    model_LGB = make_pipeline(preprocess, LGBMClassifier(objective='multiclass', random_state=5))


    # Linear Regression.

    param_grid_LGB = {#'gradientboostingregressor__max_depth': range(1,10,1), # Gradient boosting showed 3.
                      #'gradientboostingregressor__max_features':range(1,80,4)#, # 70 was normally how much it used
                      #'gradientboostingregressor__alpha':np.logspace(-2, -0.001, num=30) # 0.85
                      }

    grid_LGB = GridSearchCV(model_LGB,param_grid_LGB, cv=20)

    grid_LGB.fit(X_train, y_train)

    return grid_LGB

In [ ]:
grid_LGB = LGBM(X_train,y_train)

In [ ]:
y_pred_lgbm = grid_LGB.predict(X_test)

### Submitting

In [ ]:
pred = 0.5*y_pred_gb + 0.5*y_pred_lgbm

In [200]:
resp = toKaggle(test,pred) # We have grid_R, grid_L, grid_EN, grid_LR

In [201]:
resp.to_csv('resp_L.csv')

In [347]:
resp = toKaggle(grid_GB,test)

In [348]:
resp.to_csv('resp_GB.csv', index = False)

In [11]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from lightgbm import LGBMClassifier
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import RobustScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import GridSearchCV